# Getting images from HiCAT simulator and controlling the IrisAO

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import numpy as np
import hicat.simulators

In [ ]:
%matplotlib inline

## Instantiate hicat

In [ ]:
hc = hicat.simulators.hicat_sim.HICAT_Sim()

In [ ]:
hc.testbed_state

In [ ]:
print(hc.describe())

## Set the testbed into the correct hardware state

In [ ]:
hc.pupil_maskmask = 'circular'    # I will likely have to implement a new pupil mask
hc.iris_ao = 'iris_ao'
hc.apodizer = 'no_apodizer'
hc.lyot_stop = 'circular'
hc.detector = 'imager'

In [ ]:
hc.testbed_state

## Get an image from the simulator with flat IrisAO, unnormalized

In [ ]:
# All the images
plt.figure(figsize=(14,14))
psf, waves = hc.calc_psf(display=True, return_intermediates=True)

In [ ]:
hicat_psf = psf[0].data
print(type(hicat_psf))
print(hicat_psf.shape)

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(np.log10(hicat_psf))

## Get normalized coro PSF

### Calculate direct image

In [ ]:
hc.include_fpm = False
psf_direct_data = hc.calc_psf()

In [ ]:
psf_direct = psf_direct_data[0].data
norm = psf_direct.max()

plt.figure(figsize=(10,10))
plt.title('Direct PSF')
plt.imshow(np.log10(psf_direct), cmap='inferno')
plt.colorbar()

### Calculate normalized coro image

In [ ]:
hc.include_fpm = True
psf_coro_data = hc.calc_psf()

In [ ]:
psf_coro = psf_coro_data[0].data/norm

plt.figure(figsize=(10,10))
plt.title('Coro PSF')
plt.imshow(np.log10(psf_coro), cmap='inferno')
plt.colorbar()

## Aberrate a segment on the IrisAO

In [ ]:
hc.iris_dm.flatten()
hc.iris_dm.set_actuator(0, 50e-9, 0, 0)

In [ ]:
plt.figure(figsize=(14,14))
one_seg_data, inter = hc.calc_psf(display=True, return_intermediates=True)
one_seg = one_seg_data[0].data

In [ ]:
# Display PSF and IrisAo OPD
plt.figure(figsize=(7, 7))
plt.imshow(one_seg, norm=LogNorm(), cmap='inferno')
plt.colorbar()

In [ ]:
plt.figure(figsize=(7, 7))
inter[1].display(what='phase')

## Aberrate pair of segments

0 is the center segment

In [ ]:
hc.iris_dm.flatten()

seg1 = 4
seg2 = 23
ampl = 50e-9    # nm

hc.iris_dm.set_actuator(seg1, ampl, 0, 0)
hc.iris_dm.set_actuator(seg2, ampl, 0, 0)

In [ ]:
pair_psf_data, inter = hc.calc_psf(return_intermediates=True)
pair_psf = pair_psf_data[0].data

In [ ]:
plt.figure(figsize=(7, 7))
plt.imshow(pair_psf, norm=LogNorm(), cmap='inferno')
plt.colorbar()

In [ ]:
plt.figure(figsize=(7, 7))
inter[1].display(what='phase')

In [ ]:
hc.display_pupil_overlaps()